In [125]:
from sympy import symbols
from sympy import *

In [126]:
import numpy as np
from numpy import linalg
import scipy as sp
import math
from scipy.integrate import odeint
#from matplotlib import animation, rc
from IPython.display import HTML

import plotly.graph_objects as go
from sympy import *

In [127]:
class mugas_vibration_functions:
  
    def __init__(self): 
        self=[];

    def dynamicSystemModel(self, X, t, A, B, cParameters):
        dXdt=A @ X + B * self.controlFn(t, cParameters);
        self.X=X;
        return dXdt

    def controlFn(self, t, cParameters):
        return cParameters[0]*np.cos(cParameters[1] * t)

    def simulateDySystem(self, Tmax, dT, X0, A,B, cParameters, xLabel, yLabel):
        t = np.linspace(0, Tmax, int(Tmax/dT+1))
        sol = odeint(self.dynamicSystemModel, X0, t, args=(A, B, cParameters));
        self.soln=sol;
        fig = go.Figure()
        [nt,nS]=sol.shape;

        # Add traces
        for nn in range(nS):
            fig.add_trace(go.Scatter(x=t, y=sol[:,nn], mode='lines+markers', name='x'+str(nn+1)))
            fig.update_layout(xaxis=dict(title=xLabel),yaxis=dict(title=yLabel))
        
        fig.show()
        return [t,sol,fig] 
    
    def animate_particle_motion(self, XX, axisRange, figTitle):
        xx=[jj[0] for jj in XX]
        yy=[jj[1] for jj in XX]
        zz=[jj[2] for jj in XX]

        fig = go.Figure(
            data=[go.Scatter3d(x=[xx[0]], y=[yy[0]],z=[zz[0]],
                            mode="markers",
                            marker=dict(color="red", size=10)),
                go.Scatter3d(x=xx, y=yy,z=zz, name='Path',
                            mode="lines",
                            line=dict(color="blue", width=2))],
            layout=go.Layout(
                scene = dict(
                            xaxis=dict(range=axisRange[0], autorange=False),
                            yaxis=dict(range=axisRange[1], autorange=False),
                            zaxis=dict(range=axisRange[2], autorange=False),
                            aspectratio=dict(x=1, y=1, z=1),
                ),
                title_text=figTitle, hovermode="closest",
                updatemenus=[dict(type="buttons",
                                buttons=[dict(label="Play",
                                                method="animate",
                                                args=[None])])]),
            frames=[go.Frame(
                data=[go.Scatter3d(
                    x=[p[0]],
                    y=[p[1]],
                    z=[p[2]],
                    name='Particle',
                    mode="markers",
                    marker=dict(color="red", size=10))])

                for p in XX]
        )

        fig.show()
        return fig

    def animate_2D_scatter_plot(self, x, YY, xlabel, ylabel, title):
        fig = go.Figure(data=[go.Scatter(x=x,y=YY[0,:])],
                layout=go.Layout(
                    xaxis={'title':xlabel},
                    yaxis={'title':ylabel,'range':[1.1*YY.min(),1.1*YY.max()]},
                    title={'text':title,'y':0.9,'x':0.5, 'xanchor': 'center','yanchor': 'top'},
                    scene = dict(aspectratio=dict(x=1, y=1)),    
                    hovermode="closest",
                    updatemenus=[dict(type="buttons",
                                    buttons=[dict(label="Play",
                                                    method="animate",
                                                    args=[None])])]
                    ),
            frames=[go.Frame(data=[go.Scatter(x=x,y=y)]) for y in YY]
        )

        #fig.show()
        return fig

In [128]:
mv=mugas_vibration_functions()

In [129]:
from sympy import symbols
from sympy import *

In [130]:
m_1=1000.; m_2=1000.; k_1=10.; k_2=20.; c_1=0; c_2=0;
wn1sq=k_1/m_1;
wn2sq=k_2/m_2;
mu=m_2/m_1;
al=wn2sq/wn1sq

In [131]:
Omega=np.array([[wn1sq*(1+al*mu), -al*mu*wn1sq],[-al*wn1sq, al*wn1sq]])
S, V=linalg.eig(Omega);
w2=np.sqrt(S[0]); w1=np.sqrt(S[1]);
Xm2=V.T[0]; Xm1=V.T[1];
T=np.array([Xm1,Xm2]).T
linalg.inv(T)
gam=linalg.inv(T) @ np.array([1, 0])
gam1=gam[0]; gam2=gam[1];

In [132]:
[w1, w2]

[0.06621534468619564, 0.2135779205069857]

In [133]:
fig = go.Figure()

fig.add_trace(go.Scatter(mode="markers", x=['$x_1$','$x_2$'], y=Xm1, name=r'$\omega\approx \omega_1$' ))
fig.add_trace(go.Scatter(mode="markers", x=['$x_1$','$x_2$'], y=Xm2, name=r'$\omega\approx \omega_2$' ))

fig.update_xaxes(title='Variable') # log range: 10^0=1, 10^5=100000
fig.update_yaxes(title='Maximum Amplitude') # linear range
fig.show()

In [134]:
w=np.logspace(-1,1,1000);
Chi_x_omega=abs((-(gam1*Xm1[0]+gam2*Xm2[0])*w**2+(gam1*Xm1[0]*w2**2+gam2*Xm2[0]*w1**2))/((w1**2-w**2)*(w2**2-w**2)))/m_1;
Chi_theta_omega=abs((-(gam1*Xm1[1]+gam2*Xm2[1])*w**2+(gam1*Xm1[1]*w2**2+gam2*Xm2[1]*w1**2))/((w1**2-w**2)*(w2**2-w**2)))/m_1;

In [135]:
fig = go.Figure()

fig.add_trace(go.Scatter(mode="markers", x=w, y=Chi_x_omega, name=r'$\chi_x(\omega)$' ))
fig.add_trace(go.Scatter(mode="markers", x=w, y=Chi_theta_omega, name=r'$\chi_\theta(\omega)$' ))

fig.update_layout(xaxis=dict(type="log", title=r'$\log{\omega}$'), yaxis=dict(range=[0,10] ,title=r'$\chi(\omega)$'), title='Amplitude Frequency Response')
fig.show()

In [136]:
alPha0 = np.linspace(0, 2, 100)
mu0 = np.linspace(0, 2, 100)
alPha, mu = np.meshgrid(alPha0, mu0)
w1=0.5*(alPha*mu+alPha+1)*(1-np.sqrt(1-4*alPha/(alPha*mu+alPha+1)**2))
w2=0.5*(alPha*mu+alPha+1)*(1+np.sqrt(1-4*alPha/(alPha*mu+alPha+1)**2))

In [137]:
fig = go.Figure()

fig.add_trace(go.Surface(z=w1))
fig.add_trace(go.Surface(z=w2))

#fig.update_layout(title='Natural Frequancies', autosize=False, width=500, height=500, margin=dict(l=65, r=50, b=65, t=90))
fig.update_layout(title='Natural Frequancies', autosize=False, width=500, height=500, scene=dict(xaxis=dict(title=r'\alpha'), yaxis=dict(title=r'\mu')))

fig.show()

In [138]:
mu=.01;
alPha=np.linspace(0,2,num=100);
xx=np.sqrt(0.5*(alPha*mu+alPha+1)*(1-np.sqrt(1-4*alPha/(alPha*mu+alPha+1)**2)))
yy=np.sqrt(0.5*(alPha*mu+alPha+1)*(1+np.sqrt(1-4*alPha/(alPha*mu+alPha+1)**2)))

In [139]:
[0.20939130526150546*np.pi, 0.6753926867244588*np.pi]

[0.6578221863351233, 2.1218087029018324]

In [140]:
wn1sq=(10*np.pi)**2; 
mu=0.2;
al=1.;
Omega=np.array([[wn1sq*(1+al*mu), -al*mu*wn1sq],[-al*wn1sq, al*wn1sq]])
S, V=linalg.eig(Omega);
w2=np.sqrt(S[0]); w1=np.sqrt(S[1]);
Xm2=V.T[0]; Xm1=V.T[1];
T=np.array([Xm1,Xm2]).T
linalg.inv(T)
gam=linalg.inv(T) @ np.array([1, 0])
gam1=gam[0]; gam2=gam[1];

In [141]:
[w1, w2]

[25.16693051699425, 39.216559979075704]

In [142]:
w=np.logspace(1,2,1000);
Chi_0_omega=abs(1/((wn1sq-w**2)));
Chi_x_omega=abs((-(gam1*Xm1[0]+gam2*Xm2[0])*w**2+(gam1*Xm1[0]*w2**2+gam2*Xm2[0]*w1**2))/((w1**2-w**2)*(w2**2-w**2)));
Chi_theta_omega=abs((-(gam1*Xm1[1]+gam2*Xm2[1])*w**2+(gam1*Xm1[1]*w2**2+gam2*Xm2[1]*w1**2))/((w1**2-w**2)*(w2**2-w**2)));

In [143]:
fig = go.Figure()

fig.add_trace(go.Scatter(mode="markers", x=w, y=Chi_0_omega, name=r'$\chi_0(\omega)$' ))
fig.add_trace(go.Scatter(mode="markers", x=w, y=Chi_x_omega, name=r'$\chi_x(\omega)$' ))
fig.add_trace(go.Scatter(mode="markers", x=w, y=Chi_theta_omega, name=r'$\chi_\theta(\omega)$' ))

fig.update_layout(xaxis=dict(type="log", title=r'$\log{\omega}$'), yaxis=dict(range=[0,.04] ,title=r'$\chi(\omega)$'), title='Amplitude Frequency Response')
fig.show()

#### Low frequncy response - $\omega \approx 0$[link text](https://)

In [144]:
c_1=0; c_2=0;
A=np.array([[0,0,1,0],[0,0,0,1],[-(k_1+k_2), k_2, -(c_1+c_2), c_2],[k_2, -k_2, c_2, -c_2]]);
B=np.array([0,0,1,0]);

In [145]:
omega=0; f0=1; ICs=[0,0,0,0]
yy=mv.simulateDySystem(50,0.01,ICs,A,B,[f0,omega],'Time/t','X(t)')